In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


# Libraries

In [1]:
import os
import string
import cv2
import h5py
import fnmatch
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import backend as K
from tensorflow.keras import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, BatchNormalization, Lambda, Bidirectional, LSTM, Dense

# Data Acquisition

In [2]:
with h5py.File('mjsynth_test.hdf5', 'r') as f:
    print(list(f.keys()))
    test_img = f.get('test_img').value
    test_labels = f.get('test_labels').value

['test_img', 'test_labels']


In [3]:
test_img[0]
test_labels[0]

'kaunas'

In [4]:
char_list = string.ascii_letters + string.digits
print(char_list)
print(len(char_list))

abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789
62


In [5]:
def encode_to_labels(text):
    code = []
    for index, char in enumerate(text):
        code.append(char_list.index(char))
    return code

In [6]:
length_checker = np.vectorize(len) 
max_test_label_len = max(length_checker(test_labels))
max_label_len = max_test_label_len

Label length is the length of each output text label and input length is the same for each input to the LSTM layer which is 31 in our architecture.

In [7]:
test_label_length = []
test_input_length = []
test_txt = []

for i in range(len(test_labels)):
    test_label_length.append(len(test_labels[i]))
    test_input_length.append(31)
    test_txt.append(encode_to_labels(test_labels[i]))

In [8]:
test_padded_txt = pad_sequences(test_txt, maxlen=max_label_len, padding='post', value = len(char_list))

# Model Architecture

### Model = CNN + RNN + CTC loss

In [9]:
# input with shape of height=32 and width=128 
inputs = Input(shape=(32,128,1))
 
# convolution layer with kernel size (3,3)
conv_1 = Conv2D(64, (3,3), activation = 'relu', padding='same')(inputs)
# poolig layer with kernel size (2,2)
pool_1 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_1)
 
conv_2 = Conv2D(128, (3,3), activation = 'relu', padding='same')(pool_1)
pool_2 = MaxPool2D(pool_size=(2, 2), strides=2)(conv_2)
 
conv_3 = Conv2D(256, (3,3), activation = 'relu', padding='same')(pool_2)
 
conv_4 = Conv2D(256, (3,3), activation = 'relu', padding='same')(conv_3)
# poolig layer with kernel size (2,1)
pool_4 = MaxPool2D(pool_size=(2, 1))(conv_4)
 
conv_5 = Conv2D(512, (3,3), activation = 'relu', padding='same')(pool_4)
# Batch normalization layer
batch_norm_5 = BatchNormalization()(conv_5)
 
conv_6 = Conv2D(512, (3,3), activation = 'relu', padding='same')(batch_norm_5)
batch_norm_6 = BatchNormalization()(conv_6)
pool_6 = MaxPool2D(pool_size=(2, 1))(batch_norm_6)
 
conv_7 = Conv2D(512, (2,2), activation = 'relu')(pool_6)
 
squeezed = Lambda(lambda x: K.squeeze(x, 1))(conv_7)
 
# bidirectional LSTM layers with units=128
blstm_1 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(squeezed)
blstm_2 = Bidirectional(LSTM(128, return_sequences=True, dropout = 0.2))(blstm_1)
 
outputs = Dense(len(char_list)+1, activation = 'softmax')(blstm_2)
 
act_model = Model(inputs, outputs)

W0302 17:10:00.334715  5864 deprecation.py:506] From C:\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\ops\init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0302 17:10:01.275410  5864 deprecation.py:506] From C:\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0302 17:10:01.279437  5864 deprecation.py:506] From C:\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\ops\init_ops.py:97: calling Orthogonal.__init__ (from tensorflow.python.ops.init_ops) w

In [10]:
labels = Input(name='the_labels', shape=[max_label_len], dtype='float32')
input_length = Input(name='input_length', shape=[1], dtype='int64')
label_length = Input(name='label_length', shape=[1], dtype='int64')
 
 
def ctc_lambda_func(args):
    y_pred, labels, input_length, label_length = args
 
    return K.ctc_batch_cost(labels, y_pred, input_length, label_length)
 
 
loss_out = Lambda(ctc_lambda_func, output_shape=(1,), name='ctc')([outputs, labels, input_length, label_length])
model = Model(inputs=[inputs, labels, input_length, label_length], outputs=loss_out)

W0302 17:10:05.053905  5864 deprecation.py:323] From C:\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\ops\array_ops.py:1354: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [11]:
type(labels)

tensorflow.python.framework.ops.Tensor

In [12]:
from tensorflow.keras.callbacks import ModelCheckpoint
model.compile(loss={'ctc': lambda y_true, y_pred: y_pred}, optimizer = 'adam')

In [13]:
# load the saved best model weights
act_model.load_weights('best_model.hdf5')

# predict outputs on test images
prediction = act_model.predict(test_img)
 
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

W0302 17:18:01.313862  5864 deprecation.py:323] From C:\Anaconda3\envs\deep-learning\lib\site-packages\tensorflow\python\keras\backend.py:5438: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.


In [14]:
# see the results
i = 0
count = 0
for x in out:
    #print(valid_orig_txt[i])
    r = []
    for p in x:  
        if int(p) != -1:
            #print(char_list[int(p)], end = '')   
            r.append(char_list[int(p)])    
    #print('\n')
    print(test_labels[i],''.join(r) )
    if test_labels[i].lower() == ''.join(r).lower():
      count += 1
    i+=1
print(count,'/',i,'=',count/i)

kaunas Kaunas
Skeptical Sxsptieal
vixen VIXeN
DARKIES DARKIES
cosponsor Cipnstr
Towels Towels
SNORER SNORER
Shirttail Shirttail
FLUIDS FLUIDS
Taunted Taunted
deprecatingly deprecatingly
Cooing Caairg
Swag Swag
Fruitcakes Fuicakes
abrogator ADROeArOR
caprices caprices
jauntier iauntier
barks barks
AGNEW AENEW
BLIVET BLIVET
darling daring
sentries sentries
liquefy liquefy
CHIGNON CHIGNON
Kerosene Kerosene
seconders seconders
ORGASM ORGASM
Flt Flt
DIDOES DIDOES
diaper diapel
REQUITE ceacing
paid paid
Social Social
CORPORATE CORPORATE
WABBITS WABBITS
EXAMINE ENRTADE
talkie talkie
jibbed fibbed
PARA PARA
SCYLLA SCYLUA
false false
JEFFREY JEFFREY
ICEMAN ICEMAd
Constantly Constantly
Naughtily Maughtily
baggiest bncgiest
deflecting deflecting
paying Paying
Cheque Cheque
Roofs Roors
PROS PROS
Lr Lr
geographic geographle
SIMILES SIHiLes
GLADES GLADES
hurlers hurlers
Auspice Augpice
Botticelli Botticelli
SICKLIEST sd
Ghosting Ghosting
Smuttiness Smuttiness
SNAFU SNAFU
dogsbodies dogelodles
EVANGE

coltrane coltrane
Slaver Slaver
Magicians Magicians
upstarts upstuns
Protein Protein
Circumstantial Circumstantiol
Truer Sruer
ACTUATORS AErORS
QUALIFIERS QULCBFBERS
Nbc nle
GADGETRY CADRZITETY
Doreen Doreen
Roxy Roky
mainer gooaee
Misinformation Msinfopsatin
TONNAGES TONNACES
Marcelino Marcelino
HANCOCK Hancock
AZERBAIJANIS AZERBAJANIS
Ninths Ninths
reweave reweave
counteract counteract
UGLIER UGLIER
Bluebottle Buebottle
Gibed Bibed
Rumbustious Hmulotehiwis
motorizing motorizing
concurs concurs
Acquittal Acquittal
GROCER GROCER
CONCERTINA CONCERTINA
invoking Invoking
begrudged begrudged
Disagrees Disagreas
congaing congaing
bolder bolder
redistrict rediltriet
Whines Whines
LEDA LEDA
sedater sedater
bazaar bazaar
WHALING WIACINN
OORT Oorz
Speediest Speediest
Zoos Zoos
Shiva Sbiva
unions unions
RESOURCING RESOURCING
commuter Commuiter
INTIMATE INTIMATE
incrementally ineremenialy
wiretaps wiretaps
Machinated Machinated
pub pwb
offprint ottpxint
resown resown
KINDNESSES KINDHESSES
Marie M

Binny Birny
PYRAMIDED DRANIDED
parented Paremted
unheard upheand
minsky minsky
Herself Hanoalg
WINDER WINDER
kahuna kahuna
cowpunchers MNPNRNIY
ann alll
NOSE NOSE
Splice Spice
Scrawls Serawls
REMELTING RERELTUNG
1668 HoGe
reorders reorders
fixer fixer
OLYMPIC OLYMPIC
CORNFIELDS SEVId
Specialist Specialist
rosier rosier
Use Use
Churned Churned
Wombat Wombat
titters titters
Equipping Bqulpping
Truss Truss
retains xeteins
Magnitudes eagnitudes
uploads UPLOADS
Deplores Deplores
osculates osculates
SLANGIEST SLANGIEST
hallelujah dlald
rough rough
Violin Violin
INES INES
sanctity sunctity
goes goes
ABASHEDLY ABASHEDLY
DUBCEK DUBCEK
STOKER SICKCER
inpatients inpatients
DOWERS DOWERS
Litigious Litigious
Moo Iloa
maddox maddox
Avogadro Avogadro
Martinets Mertnets
bisect Bisett
smartly smanily
BRITONS BRITONS
PEDOPHILIA PEDOPHUA
Recreating recreating
ATTRACT ATTRAGt
Snorkel snarkel
INFALLIBLY INFALLBLY
cotters cotters
Peters Peters
WOZZECK WOZZHCK
DISORDERED DISORDERBD
PETE PETe
Reels Reele
purp

Eulipia Eulipia
Spangly Spangly
Polaris Polaris
those those
Rudder Rudder
Affable Affable
Falklands Falklands
Cinemaviva Cinemaviva
Dearly Dearly
bangladeshi bangladeshl
LITTERS LITTERS
ragout RAGOUT
MINCE AentE
Milted Milted
Vagarious Vagarious
LIBELERS LIBELERS
Shiloh Shiloh
uzis UziS
porrima porrima
tipple tipple
Teeth Testl
tongs tongs
Pealed Pealed
Giggle tiggle
hiccough hiccough
baize balze
ANUSES ANUSES
THORAZINE THORAZINE
INDICATORS IDICATORS
NIFTY NIFTY
SAVORINESS SAVORINESS
Bragg Brags
Glutinous Glutinous
sicilians Mcilians
Philippics fhlliyu
ollie ollie
Cytologist Gyologist
intestines intestines
Terahertz Senalent
earthbound earthbound
FLAGSHIPS FLAGSHIPS
FRAU FRAA
Retaliate Retaliate
POTTAGE ROTTAGE
Binderies Inbniss
rechecking rechecking
Whalers Whalers
Garde Garde
SHOWN SHOIN
INFIRM NWILRN
literalness tieraliess
Tailboard Tailboard
darkeners darkeners
Limpest Limpest
EUROPE EUROPE
outgo outgo
Added Added
Recruit Recruit
undertake undertake
porker porker
Rigging Rigging
Un

aborts aborts
ni ni
UK UK
SOAPIER SOAPIER
HORRIBLE AORRA
wafted wafted
Beam BeNN
Mammals Mmmels
CONCEITEDLY CURGEITEDLY
denizen DENZEN
Newt Newt
ELECTRON ELECtRON
MARSHALS MARSHALS
FOOLPROOF FOOLPROOF
dredgers dredgers
gold gold
Nomadic Nomadic
MARCUS MCIRGUS
ruhr ruhr
RAGGING RAGGING
repurchase epls
fumigating Fumigatng
Kegs Kegs
Craps Craps
CRAYS CRAYS
SUBEDITORS SUBEOTORS
vector vector
Delves Delves
tiddlers tiddlers
untimely UNTIMELY
ghz ghz
SUNDERS SUNDERS
Weatherman Weatherman
Riddance RiDDnce
embroidering embraidering
Undress Undress
PREPAY PREPAY
hying hying
Eyeballs Eyeballs
Nauseousness Nauseousness
hoosier hoosier
DEVIATES Aefaagey
savored savored
wheeled wheeled
Latinas Latinas
OUTWITS OUrwsfs
orderliness ordurliness
ARTS AKtE
Dew Devn
huntsville huntsville
mouthpiece MEMTIOUl
Clueless Clueless
Untested Lntested
Spookiness SRooe
mailings mailings
decals decals
QUIP Qulp
lewis Lemis
guidance guidance
Backlog Backlog
Mortar Mortan
Embargoed Enbageed
item item
Weepie Weepis
cl

...

CAPSIZING CAPSTZING
Catechesis Catechests
CRIPPLING CRIPPLIN
overweight ovenweight
ARIDITY AOT
Interstellar Interstellar
CPR CPR
CADETS CADETS
ttys Glys
APPENDING AFPEfHII
Cataloging Catsloging
Tankful FanKfut
grunted grunted
Decapitating Decapitating
Citizens Citizens
bulk luk
ARMANI ReinGnd
gena gena
oppressed oppressed
thereat thereat
flurry flurry
Euphoric euphorie
Safeway Safeway
BITMAP BITMAP
negotiation negotiation
MUNICH MONUCiE
TENEMENT TENEMENT
iga iga
doses doses
Bullets Buillets
CLEAVED CLEAVED
headlamp headamp
Bedstead Bedstead
temples temples
airliners airliners
Dizzier Dizzier
impeccably IMRELUADY
marches marches
Clobbering clobbating
sterile sterile
Webbing Wehhisn
Nerds Nerds
imprint imprint
ravishers ravishers
Pedophile Bedophile
Fining Fining
Unreeling Unreeling
baikal hunlenil
ersatz ersatz
dismay dismay
Lateraling Lateraling
cavorts Bavexr
puds puds
oversteps oversteps
primulas primules
linda linda
Tetanus Tetanus
annihilating mnniblating
Bereaving Bereavin
Batty B

drollery drollery
CAMPUS CAMPUS
Astutely Astutely
Zippiest Iippiest
pizzicato pizzicato
CARR CARR
Interviews Interviews
Understandingly sndersiandingl
Tutto Turre
Bonk Bonk
asphyxiated BAOIADLS
Intoned Intoned
skirting Skiting
Calipered Calipered
Callosities Cullasities
Spinnakers Spinnakers
backfiring hackfiving
doweling Jowaling
SHIRKER SHIRKER
excite excfte
PLIANTLY PLIRNTLY
QUOTABILITY QUOTABILITY
Traumatizing Traumatizing
Mildews Midews
VACUOUS VACUOUS
Yearlies Yearlies
calcifying calfying
oscillation oscilation
plainclothes plainclothes
hatchback hatchback
HYPING APING
Tomtits Tintrs
Snippet Snippet
burled buuled
YUAN YUAN
Conceptualize Canceptvalize
causers causers
spill spill
Uprights Upriohts
sunbath subat
ILLICITNESS HACTNeS
SNICKED SNBKED
OMAHA OmAHe
finger finger
EXISTING BXSTING
Pillorying Pilorying
PETRIFY PETRIFY
FLUENT FLUENT
doing doing
unfaltering unfaltering
Clubbed Cubbed
Summers Summners
Wreath Wreath
Coolies Coolies
Dabblers pabblers
HOSTS HOSTS
Crater Grater
cras

bi Bl
LASTING LASFING
Rabelais Rabelais
RESIN RESIN
RUMS RUMS
TAMPS TAMPS
Inflected Inflected
algieba alieha
TORQUED TOROUED
PICCOLO PICCOLO
Shorthand Shorthand
Presently presenitly
SINFULLY SINFULLY
Tuns funs
DELLA VELLA
Ionized lonized
Fulls Fulls
MAIMING MNiNG
Chown Chown
gefer GEFER
SPOOFED FOOREID
handsprings Mandsprings
naphtali naphtall
Spacewalks STUKkn
Forgive Forgive
Maltose Maltose
moderators moderators
SEABIRDS SEABIRDS
Hummock Mummock
Shiploads Shiploads
LETTERS LETTERS
lappet lappet
exulting AULTLNG
MIMING HWING
CLERESTORY CLERESTORY
Ortiz ortiz
cadres cadres
dabbed dabbed
Prov Puoy
Clammier Clammier
RUNABOUTS RUNABOUTS
albanians albanians
Bassets BABSETS
ILLEGIBLE JReGIBRE
Alcibiades Ncbiacles
ilia ilia
fums daas
Musket Llasel
chinning chinning
SPOILAGE SPOLAGE
heightens haiohtens
9 S
INKED INKED
Yaws Yaws
WEISS WESS
KNOLL KNOLL
arce aree
SPORTSCAST SPORTSCAST
bandung bandung
feather feather
Dutiful Dudjful
Swoon Suoon
eviscerating Gvisceratine
GROWLS GROWLS
canaanites c

fitness fitness
JODHPURS JODHURS
BERENICE BERENICE
Interminable Interminable
Bonny Bonny
flabbergasting deblwteting
Nurture Nurture
uncover uncover
REFUELS REFUELS
Lexus Lexus
MUSETTES HUSETTES
Advocating Cdooeetins
YOKED QOXED
COASTING COASTING
hematology hematology
GILL GILL
chamomiles Eofomtjes
FOLLOWER FOLLOWER
TENTHS TENTHS
SHINS SHINS
Coverings Borig
TARSALS TARSALS
minims minims
MOSUL MOSUL
platting platfing
plusher plusher
Blinked Blinked
Karla Karla
Subgroup Subgroup
Agilest Agilest
Trappists Trappists
partying partying
dybbuk dybbuk
dissipates dissipates
Waylays Llugiugs
Disinherit minkerit
psycho psycho
FLYWHEEL ELewRBEt
COTTONMOUTHS COtToNMOUTHS
teaism SEAUSN
cue CUE
amok amok
pan par
Turmoil Turwoil
muddies nuddies
SQUISHED SQUISHED
Prance Fra
Clotho Clotho
Attachment ARtachment
bangalore bangalore
NADIRS IROKRS
VITIS VIS
Keyboarders Rerardeis
dampener lanpener
paradise Paradise
dowry dowry
coffin coffin
FATIMA FATIMA
Girdling Girdlina
tougheners fougheners
Chewy Chewy
mig

fidgeting fidgeting
schmo schmo
Criminology Crimninolosy
Fiercer Fercer
SMUGGER SMUGCER
Resenting Reseniting
Stub geab
Songfest Songfet
Zane Zane
smites smites
Inguinal Inguinal
Cravens Cravens
hearten hearten
hyperion huparlon
Farsighted Farsighted
STRETCHER StRetCHER
HAMPER HAMPER
slurping slurping
weakling mcallig
Spattering patlering
cattlemen Cattlamen
Slender Slender
Intoners IOIOKK
BOOKSHELVES BAPHSHENES
Boyle Boyle
Notice NiKe
acknowledgments acdnoslictmens
slow slow
Escalloped Escalloped
Casabas Casabas
DISPERSED DISPERSED
torment tormant
WARDRESS WARDRESS
trevino trevino
tingling fingling
duskiest duskiest
housemother housemcter
Amity Amity
TOWERS TOUERS
Devises Devises
KINKS KINKS
HIRSUTE HIRSUTE
Pointier POnber
Defiling Defiling
PHASES FAses
Ail Ail
faves faves
Dubhe Dubhe
aida aida
lipreading ireading
tim tim
Macadamizing Macadamizing
discoed discced
besoms eeeoae
cravenness BuayEneS
PUSTULE PUSTULE
Colson Coison
Uninstalled wninstalled
HELPINGS HELPIGS
Nitpickers flsplons

## Check for dynamically uploaded image

In [15]:
addr = 'test/141_Correspondents_17170.jpg'
test_label = "Correspondents"

img = cv2.imread(addr)
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

scale_percent = 50

#calculate the 50 percent of original dimensions
width = 32
height = 128

# dsize
dsize = (height, width)
img = cv2.resize(img, dsize)
img = np.expand_dims(img , axis = 2)
img = img/255.

In [16]:
img.shape

(32, 128, 1)

In [17]:
# load the saved best model weights
act_model.load_weights('best_model.hdf5')

# predict outputs on validation images
prediction = act_model.predict(img[np.newaxis,:,:])
 
# use CTC decoder
out = K.get_value(K.ctc_decode(prediction, input_length=np.ones(prediction.shape[0])*prediction.shape[1],
                         greedy=True)[0][0])

In [18]:
# see the results
count = 0
for x in out:
    #print(valid_orig_txt[i])
    r = []
    for p in x:  
        if int(p) != -1:
            #print(char_list[int(p)], end = '')   
            r.append(char_list[int(p)])    
    #print('\n')
    print(test_label,''.join(r) )

Correspondents Correspondents
